### Data Processing and Storage

#### 01 - Import dataset from RDS

In [1]:
import boto3
import pandas as pd
from rds import create_rds_engine, get_rds_instance
from env import (
    AWS_PROFILE_NAME,
    RDS_DB_NAME,
    RDS_INSTANCE_NAME,
    RDS_PASSWORD,
    RDS_USER,
    DEFAULT_RDS_DB_TABLE,
)

profile_name = AWS_PROFILE_NAME
master_username = RDS_USER
master_password = RDS_PASSWORD
db_name = RDS_DB_NAME
db_instance_identifier = RDS_INSTANCE_NAME
default_table = DEFAULT_RDS_DB_TABLE

boto_session = boto3.Session(profile_name=profile_name)

rds_client = boto_session.client("rds")

endpoint = get_rds_instance(rds_client, db_instance_identifier)

engine = create_rds_engine(master_username, master_password, endpoint, db_name)

query = f"SELECT * FROM `{default_table}`"

df = pd.read_sql(query, con=engine)

RDS Instance founded


#### 02 - Add necessary column label_avg_price_per_room in dataset for data processing

In [2]:
def generate_avg(avg_price_per_room: int):
    if avg_price_per_room <= 85:
        return 1
    elif avg_price_per_room < 115:
        return 2
    else:
        return 3

df["label_avg_price_per_room"] = df["avg_price_per_room"].apply(generate_avg)

df

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status,label_avg_price_per_room
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled,1
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled,2
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled,1
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled,2
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,INN36271,3,0,2,6,Meal Plan 1,0,Room_Type 4,85,2018,8,3,Online,0,0,0,167.80,1,Not_Canceled,3
36271,INN36272,2,0,1,3,Meal Plan 1,0,Room_Type 1,228,2018,10,17,Online,0,0,0,90.95,2,Canceled,2
36272,INN36273,2,0,2,6,Meal Plan 1,0,Room_Type 1,148,2018,7,1,Online,0,0,0,98.39,2,Not_Canceled,2
36273,INN36274,2,0,0,3,Not Selected,0,Room_Type 1,63,2018,4,21,Online,0,0,0,94.50,0,Canceled,2


#### 03 - Transformation of categorical variables with dummies

In [3]:
df = pd.get_dummies(
    data=df,
    prefix=[
        "type_of_meal_plan",
        "room_type_reserved",
        "market_segment_type",
    ],
    columns=[
        "type_of_meal_plan",
        "room_type_reserved",
        "market_segment_type",
    ],
)

#### 04 - Drop columns after analyses

In [4]:
df.drop(
    columns=[
        "avg_price_per_room",
        "Booking_ID",
        "booking_status",
        "no_of_weekend_nights",
        "no_of_week_nights",
        "repeated_guest",
        "no_of_previous_cancellations",
        "no_of_previous_bookings_not_canceled",
    ],
    inplace=True,
)

#### 05 - Convert categorical variables with dummies in numeric variables   

In [5]:
df = df * 1

#### 06 - Create and insert processed dataset in a new table in RDS

In [6]:
df.to_sql(f"{default_table}-processed", con=engine, if_exists="replace", index=False)

36275